In [1]:
import json

In [2]:
from pathlib import Path

In [3]:

path = Path("../data/challenge-2-dataset-and-documentation/dataset/train/output/sources_info/sources_info.json")

In [4]:
path

PosixPath('../data/challenge-2-dataset-and-documentation/dataset/train/output/sources_info/sources_info.json')

In [5]:
with open(path, "r") as f:
    annotation_data = json.loads(f.read())

In [6]:
labels = annotation_data["data"]

In [7]:
main_docs_path = Path("../data/platform-docs-versions/")

In [9]:
# for doc in labels:
#     print(doc["idx"])
#     for section in doc["sections"]:
#         path = section["path"]
#         full_path = main_docs_path / path
#         print(full_path.is_file(), full_path)
#     print()
#     # print(doc)

In [34]:
# API Documentation.md

In [10]:
from pprint import pprint

In [11]:
for doc in 

{'data': [{'idx': 0,
           'sections': [{'lines': [],
                         'path': 'Facebook_API-Ad/API Meta Ad Library.md'},
                        {'lines': [],
                         'path': 'Facebook_Graph-API/Ads Archive '
                                 'Endpoint.md'}]},
          {'idx': 1,
           'sections': [{'lines': [],
                         'path': 'Facebook_api-Ad/API Meta Ad Library.md'}]},
          {'idx': 2,
           'sections': [{'lines': [],
                         'path': 'platform-docs-declarations/platforms.md'}]},
          {'idx': 3,
           'sections': [{'lines': [],
                         'path': 'Linkedin_Government-Requests/Government '
                                 'Request Report.md'}]},
          {'idx': 4,
           'sections': [{'lines': [],
                         'path': 'Pinterest_DSA-Report/DSA Report.md'}]},
          {'idx': 5,
           'sections': [{'lines': [],
                         'path': 'CrowdTangle_API/